# Инфраструктура для моделей машинного обучения. Практическая работа

# Цель практической работы

Потренироваться в использовании библиотек PySpark SQL и PySpark ML для предобработки данных и обучения моделей.

# Что входит в практическую работу

1. Инициализация спарк-сессии.
2. Загрузка данных.
3. Ознакомление с данными.
4. Преобразование типов столбцов.
5. Очистка данных.
6. Feature-инжиниринг.
7. Векторизация фичей.
8. Создание и обучение модели.
9. Выбор лучшей модели.
10. Обратная связь.


# Что оценивается

- Пройдены все этапы работы.
- Спарк-сессия успешно запущена.
- Данные прочитаны.
- Все колонки с числовыми значениями преобразованы в числовые типы данных (Int или Double).
- Отфильтрованы все строки с Null-значениями.
- Созданы новые фичи.
- Все категориальные колонки преобразованы в числовой вид, выполнены все этапы векторизации признаков.
- Выборка разделена на обучающую и тестовую.
- Создано три объекта: модель, сетка гиперпараметров и evaluator.
- Создан объект класса CrossValidator и обучен на обучающей выборке.
- Выбрана лучшая модель, посчитана метрика качества лучшей модели.


# Задача

Используя данные о клиентах телекоммуникационной компании, обучите модель, предсказывающую их отток.

Описание данных, с которыми вы будете работать:

* **CustomerID**: ID клиента.
* **Gender**: пол клиента.
* **SeniorCitizen**: пенсионер ли клиент (1 — да, 0 — нет).
* **Partner**: есть у клиента партнёр (жена, муж) или нет (Yes/No).
* **Dependents**: есть ли у клиента инждивенцы, например дети (Yes/No).
* **Tenure**: как много месяцев клиент оставался в компании.
* **PhoneService**: подключена ли у клиента телефонная служба (Yes/No).
* **MultipleLines**: подключено ли несколько телефонных линий (Yes, No, No phone service).
* **InternetService**: интернет-провайдер клиента (DSL, Fiber optic, No).
* **OnlineSecurity**: подключена ли у клиента услуга онлайн-безопасности (Yes, No, No internet service)
* **OnlineBackup**: подключена ли услуга резервного копирования онлайн (Yes, No, No internet service).
* **DeviceProtection**: подключена ли услуга защиты устройства (Yes, No, No internet service)
* **TechSupport**: есть ли у клиента техническая поддержка (Yes, No, No internet service).
* **StreamingTV**: подключена ли услуга потокового телевидения (Yes, No, No internet service).
* **StreamingMovies**: подключена ли услуга стримингового воспроизведения фильмов (Yes, No, No internet service).
* **Contract**: тип контракта клиента (Month-to-month, One year, Two year).
* **PaperlessBilling**: есть ли безбумажный счёт.
* **PaymentMethod**: способ оплаты услуг (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)).
* **MonthlyCharges**: сумма, которая списывается ежемесячно.
* **TotalCharges**: сумма, списанная за всё время.
* **Churn**: ушёл ли клиент (Yes/No). Это целевая переменная, которую нужно предсказать.


# 1. Инициализация спарк-сессии

Инициализируйте спарк-сессию.

Эта ячейка нужна для того, чтобы заргузить необходимые библиотеки и настроить окружение Google Colab для работы со Spark.

Просто запустите её перед выполением задания :)

In [4]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')


In [5]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("Churn Prediction").getOrCreate()

# 2. Загрузка данных
Загрузите данные, сохраните их в переменную типа sparkDataframe, используя метод read.csv (не забывайте про header и delimiter).

In [6]:
df = spark.read.csv(
    "WA_Fn-UseC_-Telco-Customer-Churn.csv", header=True, inferSchema=True
)

# 3. Ознакомление с данными
1. Выведите на экран первые несколько строк датафрейма.


In [7]:
df.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    


2. Выведите общее количество строк датафрейма.



In [8]:
df.count()

7043

3. Выведите структуру (схему) датафрейма.

In [9]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



# 4. Преобразование типов столбцов
Преобразуйте тип столбцов у числовых признаков (Int — если признак целочисленный, Double — если признак не целочисленный). Сохраните преобразованный датафрейм в новую переменную.

## Совет

Если вам сложно выполнить это задание, изучите дополнительные материалы: [об операторе Cast](https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/), [об операторе Select](https://sparkbyexamples.com/pyspark/select-columns-from-pyspark-dataframe/).



In [12]:
from pyspark.sql.functions import expr, col

for column in ["SeniorCitizen", "Tenure"]:
    df1 = df.withColumn(column, col(column).cast("int"))

for column in ["MonthlyCharges", "TotalCharges"]:
    df1 = df1.withColumn(column, col(column).cast("double"))

In [13]:
df1.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



# 5. Очистка данных
Проверьте, есть ли в какой-либо колонке Null-значения. Для этого можно использовать your_dataframe.filter(col("colname")).isNull()).

Выведите на экран несколько строк с Null-значениями в одной из колонок.

Сохраните очищенный от строк с Null-значениями датафрейм в новую переменную. Для фильтрации этих значений можно использовать метод .isNotNull().

Колонок в датафрейме много, проверять каждую неудобно и долго. Подумайте, как упроситить эту работу, если использовать, например, перебор с циклом for.

[Примеры использования операторов isNull() и isNotNull()](https://sparkbyexamples.com/pyspark/pyspark-isnull/).


In [14]:
null_counts = {}

for column in df1.columns:
    null_count = df1.filter(col(column).isNull()).count()
    null_counts[column] = null_count

for column, count in null_counts.items():
    print(f"Column: {column}, Null Count: {count}")

Column: customerID, Null Count: 0
Column: gender, Null Count: 0
Column: SeniorCitizen, Null Count: 0
Column: Partner, Null Count: 0
Column: Dependents, Null Count: 0
Column: Tenure, Null Count: 0
Column: PhoneService, Null Count: 0
Column: MultipleLines, Null Count: 0
Column: InternetService, Null Count: 0
Column: OnlineSecurity, Null Count: 0
Column: OnlineBackup, Null Count: 0
Column: DeviceProtection, Null Count: 0
Column: TechSupport, Null Count: 0
Column: StreamingTV, Null Count: 0
Column: StreamingMovies, Null Count: 0
Column: Contract, Null Count: 0
Column: PaperlessBilling, Null Count: 0
Column: PaymentMethod, Null Count: 0
Column: MonthlyCharges, Null Count: 0
Column: TotalCharges, Null Count: 11
Column: Churn, Null Count: 0


In [15]:
df_clean = df1.na.drop()

# 6. Feature-инжиниринг
Добавьте в датафрейм одну или несколько новых фичей. Удалите колонки, которые, как вам кажется, нужно убрать из фичей. Обоснуйте свои решения.

In [16]:
from pyspark.sql.functions import when

df_clean = df_clean.withColumn(
    "TotalCharges_Per_Month", col("TotalCharges") / col("Tenure")
)
df_clean = df_clean.drop("CustomerID", "TotalCharges")

# 7. Векторизация фичей
Подготовьте данные к обучению:





1. Преобразуйте текстовые колонки в числа, используя StringIndexer.
Удалите столбцы со старыми (непреобразованными) признаками. Выведите на экран структуру получившегося датафрейма. Не забывайте о столбце Churn. Хоть он и выступает в задаче как таргет, он имеет текстовый тип, поэтому тоже должен быть закодирован числовыми значениями.

Чтобы использовать StringIndexer для всех категориальных признаков сразу, а не для каждого отдельно, можно применить сущность pipeline.

**Пример кода:**

##### #Задаём список текстовых колонок:
text_columns = ["text_col_1", "text_col_2", "text_col_3"]

##### #Задаём список StringIndexer'ов — сущностей, каждая из которых будет кодировать одну текстовую колонку числами. Имена преобразованных колонок будут заканчиваться на _index:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(<ваш датасет>) for column in text_columns]

##### #Создаём Pipeline из StringIndexer'ов:
pipeline = Pipeline(stages=indexers)

##### #Скармливаем нашему pipeline датафрейм, удаляя старые колонки:
new_dataframe = pipeline.fit(<ваш датасет>).transform(<ваш датасет>).drop(*text_columns)


In [17]:
from pyspark import mllib
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

# список колонок с текстовым типом
text_cols = [
    "gender",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "Churn",
]


indexers = [
    StringIndexer(inputCol=column, outputCol=column + "_index").fit(df_clean)
    for column in text_cols
]

pipeline = Pipeline(stages=indexers)
model = pipeline.fit(df_clean)
df_indexed = model.transform(df_clean)
df_indexed = df_indexed.drop(*text_cols)

2. Векторизуйте категориальные признаки, используя OneHotEncoder.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран структуру получившегося после преобразований датафрейма.


In [18]:
from pyspark.ml.feature import OneHotEncoder

# список категориальных колонок
features_inp = [
    "gender",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
]


categorical_cols = [(col + "_index") for col in features_inp]
encoder = OneHotEncoder(
    inputCols=categorical_cols, outputCols=[col + "_ohe" for col in categorical_cols]
)
df_encoded = encoder.fit(df_indexed).transform(df_indexed).drop(*categorical_cols)

df_encoded.printSchema()

root
 |-- SeniorCitizen: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges_Per_Month: double (nullable = true)
 |-- Churn_index: double (nullable = false)
 |-- gender_index_ohe: vector (nullable = true)
 |-- Partner_index_ohe: vector (nullable = true)
 |-- Dependents_index_ohe: vector (nullable = true)
 |-- PhoneService_index_ohe: vector (nullable = true)
 |-- MultipleLines_index_ohe: vector (nullable = true)
 |-- InternetService_index_ohe: vector (nullable = true)
 |-- OnlineSecurity_index_ohe: vector (nullable = true)
 |-- OnlineBackup_index_ohe: vector (nullable = true)
 |-- DeviceProtection_index_ohe: vector (nullable = true)
 |-- TechSupport_index_ohe: vector (nullable = true)
 |-- StreamingTV_index_ohe: vector (nullable = true)
 |-- StreamingMovies_index_ohe: vector (nullable = true)
 |-- Contract_index_ohe: vector (nullable = true)
 |-- PaperlessBilling_index_ohe: vector (nullable = true)
 |-- Payment

3. Объедините колонки фичей в один вектор, используя VectorAssembler.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран первые несколько строк и структуру получившегося датафрейма.

In [19]:
from pyspark.ml.feature import VectorAssembler


feature_cols = [col + "_ohe" for col in categorical_cols]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_final = assembler.transform(df_encoded)

In [20]:
df_final.show(5)
df_final.printSchema()

+-------------+------+--------------+----------------------+-----------+----------------+-----------------+--------------------+----------------------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+------------------+--------------------------+-----------------------+--------------------+
|SeniorCitizen|Tenure|MonthlyCharges|TotalCharges_Per_Month|Churn_index|gender_index_ohe|Partner_index_ohe|Dependents_index_ohe|PhoneService_index_ohe|MultipleLines_index_ohe|InternetService_index_ohe|OnlineSecurity_index_ohe|OnlineBackup_index_ohe|DeviceProtection_index_ohe|TechSupport_index_ohe|StreamingTV_index_ohe|StreamingMovies_index_ohe|Contract_index_ohe|PaperlessBilling_index_ohe|PaymentMethod_index_ohe|            features|
+-------------+------+--------------+----------------------+-----------+----------------+-----------------+-----------------

# 8. Создание и обучение модели

1. Создайте модель — логистическую регрессию (используя LogisticRegression). В качестве параметров класса LogisticRegression укажите колонку фичей (параметр featuresCol), колонку-таргет (параметр labelCol) из датафрейма и имя колонки, в которую будут записываться предсказания (параметр predictionCol).

In [21]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="Churn_index")

2. Разделите датафрейм на обучающую и тестовую выборку.

In [22]:
train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=1234)

3. Создайте объекты — сетки гиперпараметров для каждой модели, используя ParamGridBuilder. Так же, как и в ноутбуке из последнего видео, в сетку гиперпараметров можно добавить значения параметров regParam и elasticNetParam.

Вы можете ознакомиться [с документацией объекта LogisticRegression в PySpark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html) и добавить в сетку больше параметров.


In [23]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = (
    ParamGridBuilder()
    .addGrid(lr.regParam, [0.1, 0.01])
    .addGrid(lr.elasticNetParam, [0.0, 1.0])
    .build()
)

4. Создайте объект evaluator, который будет отвечать за метрику качества при обучении. Для этого используйте класс BinaryClassificationEvaluator со следующими параметрами: rawPredictionCol — колонка с предсказаниями, labelCol — колонка с таргетом.

У вас, возможно, возник вопрос, какую метрику качества берёт по умолчанию BinaryClassificationEvaluator. По умолчанию BinaryClassificationEvaluator будет рассчитывать areaUnderROC. Это метрика оценки площади под кривой ROC (Receiver Operating Characteristic), которая служит графической интерпретацией производительности модели. Эта метрика качества находится в пределах от 0 до 1. Чем выше метрика, тем более качественные предсказания делает модель.

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="prediction", labelCol="Churn_index"
)

5. Создайте объект CrossValidator, в качестве параметров укажите уже созданные вами модель, сетку гиперпараметров и evaluator.

In [25]:
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(
    estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3
)

6. Запустите обучение модели на тренировочной выборке. Сохраните обученную модель в новую переменную.

In [26]:
cvModel = cv.fit(train_df)

# 9. Выбор лучшей модели

1. Выберите лучшую модель, сохраните её в отдельную переменную, отобразите её параметры.

Вывод параметров модели в PySpark можно сделать, используя метод extractParamMap().

In [27]:
best_model = cvModel.bestModel
best_model.extractParamMap()

{Param(parent='LogisticRegression_763b36de2882', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_763b36de2882', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_763b36de2882', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_763b36de2882', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LogisticRegression_763b36de2882', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_763b36de2882', name='labelCol', doc='label column name.'): 'Churn_index',
 Param(parent='LogisticRegression_763b36de2882', name='maxBlockSizeInMB', doc='maximum memory in MB

2. Запустите лучшую модель в режиме предсказания на тестовой выборке. Сохраните предсказания в отдельную переменную. Выведите первые несколько строк датафрейма с предсказаниями на экран.

Запуск модели в режиме предсказания выполняется при помощи метода .transform(<тестовая выборка>).

In [28]:
predictions = best_model.transform(test_df)
predictions.show()

+-------------+------+--------------+----------------------+-----------+----------------+-----------------+--------------------+----------------------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+------------------+--------------------------+-----------------------+--------------------+--------------------+--------------------+----------+
|SeniorCitizen|Tenure|MonthlyCharges|TotalCharges_Per_Month|Churn_index|gender_index_ohe|Partner_index_ohe|Dependents_index_ohe|PhoneService_index_ohe|MultipleLines_index_ohe|InternetService_index_ohe|OnlineSecurity_index_ohe|OnlineBackup_index_ohe|DeviceProtection_index_ohe|TechSupport_index_ohe|StreamingTV_index_ohe|StreamingMovies_index_ohe|Contract_index_ohe|PaperlessBilling_index_ohe|PaymentMethod_index_ohe|            features|       rawPrediction|         probability|prediction|
+-------------+---

3. Получите метрику качества модели. Для этого примените к объекту evaluator метод .evaluate(<ваш датафрейм с предсказаниями>).



In [29]:
accuracy = evaluator.evaluate(predictions)
print("Test Area Under ROC: ", accuracy)

Test Area Under ROC:  0.6917851701371255


# 10. Обратная связь
Вы ознакомились с возможностями двух мощных библиотек: PySpark SQL для предобработки данных и PySpark ML для машинного обучения.

Поделитесь впечатлениями от работы с новыми библиотеками. В чём они более удобны, чем уже знакомые вам Pandas и Sklearn, а в чём нет.

Синтаксис непривычен (тот же `random_state` обозвали как `seed`).

В целом, удобно, что есть много готовых решений.

# Как отправить работу на проверку

Загрузите файл с заданиями, откройте его через Jupyter Notebook в Google Colab. Скачайте файл с датасетом и загрузите его в Colab. Выполните задачи, сохраните изменения: воспользуйтесь опцией Save and Checkpoint из вкладки меню File или кнопкой Save and Checkpoint на панели инструментов. Отправьте через форму ниже итоговый файл Jupyter Notebook (.ipynb) или ссылку на него.